In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('X_test.csv').drop(['ID', 'Day', 'Minute'], axis=1)

In [3]:
y = train['TARGET']
x = train.drop(['TARGET', 'Day', 'Minute'], axis=1)

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
test = sc.transform(test)

In [5]:
x_tensor = torch.FloatTensor(x)
y_tensor = torch.FloatTensor(y).reshape(-1, 1)
test_tensor = torch.FloatTensor(test)

In [6]:
layer1 = torch.nn.Linear(x_tensor.shape[1], 16)
layer2 = torch.nn.Linear(16, 32)
layer3 = torch.nn.Linear(32, 64)
layer4 = torch.nn.Linear(64, 32)
layer5 = torch.nn.Linear(32, 16)
layer6 = torch.nn.Linear(16, 1)
model = torch.nn.Sequential(
    layer1,
    torch.nn.Sigmoid(),
    layer2,
    torch.nn.Sigmoid(),
    # layer3,
    # torch.nn.Sigmoid(),
    # layer4,
    # torch.nn.Sigmoid(),
    layer5,
    torch.nn.Sigmoid(),
    layer6
)

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [8]:
epochs = 15000

for epoch in range(epochs):
    optimizer.zero_grad()
    hypothesis = model(x_tensor)
    cost = torch.nn.functional.mse_loss(hypothesis, y_tensor)
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'epoch: {epoch}, cost: {cost.item()}')

epoch: 0, cost: 1011.8375854492188
epoch: 100, cost: 89.71498107910156
epoch: 200, cost: 27.479379653930664
epoch: 300, cost: 17.581758499145508
epoch: 400, cost: 13.2183198928833
epoch: 500, cost: 10.66579818725586
epoch: 600, cost: 8.805891990661621
epoch: 700, cost: 7.190576076507568
epoch: 800, cost: 7.3705267906188965
epoch: 900, cost: 3.803879976272583
epoch: 1000, cost: 5.936590671539307
epoch: 1100, cost: 4.274758815765381
epoch: 1200, cost: 4.462921619415283
epoch: 1300, cost: 4.282958984375
epoch: 1400, cost: 4.183465480804443
epoch: 1500, cost: 4.153087615966797
epoch: 1600, cost: 4.145956039428711
epoch: 1700, cost: 4.0149617195129395
epoch: 1800, cost: 3.835703134536743
epoch: 1900, cost: 3.659553050994873
epoch: 2000, cost: 3.5047218799591064
epoch: 2100, cost: 3.3795106410980225
epoch: 2200, cost: 3.314357280731201
epoch: 2300, cost: 3.1321377754211426
epoch: 2400, cost: 3.0554099082946777
epoch: 2500, cost: 2.9799156188964844
epoch: 2600, cost: 2.9108822345733643
epoch:

In [9]:
predict = model(test_tensor).detach().numpy()
submit = pd.read_csv('sample_submit.csv')
submit['TARGET'] = predict
submit.to_csv('submit.csv', index=False)